In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import bambi as bmb
import kulprit as kpt

import arviz as az
import matplotlib.pyplot as plt

In [3]:
data = pd.read_stata("https://stats.idre.ucla.edu/stat/stata/dae/nb_data.dta")
data["prog"] = data["prog"].map({1: "General", 2: "Academic", 3: "Vocational"})
data.head()

,id,gender,math,daysabs,prog
0,1001.0,male,63.0,4.0,Academic
1,1002.0,male,27.0,4.0,Academic
2,1003.0,female,20.0,2.0,Academic
3,1004.0,female,16.0,3.0,Academic
4,1005.0,female,2.0,3.0,Academic


In [4]:
model = bmb.Model("daysabs ~ prog + scale(math)", data, family="poisson")
idata = model.fit()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Intercept, prog, scale(math)]
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


/Users/yannmclatchie/Desktop/kulprit/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/yannmclatchie/Desktop/kulprit/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/yannmclatchie/Desktop/kulprit/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/yannmclatchie/Desktop/kulprit/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 22 seconds.


In [98]:
new_idata = idata
new_idata.posterior

<xarray.Dataset>
Dimensions:      (chain: 4, draw: 1000, prog_dim: 2)
Coordinates:
  * chain        (chain) int64 0 1 2 3
  * draw         (draw) int64 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * prog_dim     (prog_dim) <U10 'General' 'Vocational'
Data variables:
    Intercept    (chain, draw) float64 1.902 1.904 1.894 ... 1.873 1.888 1.852
    prog         (chain, draw, prog_dim) float64 0.4747 -0.9088 ... -0.9203
    scale(math)  (chain, draw) float64 -0.1667 -0.1411 ... -0.1457 -0.1793
Attributes:
    created_at:                  2022-10-19T10:31:00.217778
    arviz_version:               0.12.1
    inference_library:           pymc
    inference_library_version:   4.0.0
    sampling_time:               21.843305826187134
    tuning_steps:                1000
    modeling_interface:          bambi
    modeling_interface_version:  0.9.0

In [106]:
new_idata.posterior

<xarray.Dataset>
Dimensions:      (chain: 4, draw: 1000, prog_dim_0: 2)
Coordinates:
  * chain        (chain) int64 0 1 2 3
  * draw         (draw) int64 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
    prog_dim     (prog_dim_0) <U10 'General' 'Vocational'
Dimensions without coordinates: prog_dim_0
Data variables:
    Intercept    (chain, draw) float64 1.902 1.904 1.894 ... 1.873 1.888 1.852
    prog         (chain, draw, prog_dim_0) float64 0.4747 -0.9088 ... -0.9203
    scale(math)  (chain, draw) float64 -0.1667 -0.1411 ... -0.1457 -0.1793
Attributes:
    created_at:                  2022-10-19T10:31:00.217778
    arviz_version:               0.12.1
    inference_library:           pymc
    inference_library_version:   4.0.0
    sampling_time:               21.843305826187134
    tuning_steps:                1000
    modeling_interface:          bambi
    modeling_interface_version:  0.9.0

In [100]:
new_idata.rename_dims({"prog_dim": "prog_dim_0"}, inplace=True)

In [105]:
idata.posterior.dims

Frozen({'chain': 4, 'draw': 1000, 'prog_dim_0': 2})

In [ ]:

new_idata.rename_dims()

In [103]:
ref_model = kpt.ReferenceModel(model, idata)
ref_model.search()

Frozen({'chain': 4, 'draw': 1000, 'prog_dim_0': 2})
Frozen({'chain': 4, 'draw': 100})
Frozen({'chain': 4, 'draw': 1000, 'prog_dim_0': 2})
Frozen({'chain': 4, 'draw': 100})
Frozen({'chain': 4, 'draw': 1000, 'prog_dim_0': 2})
Frozen({'chain': 4, 'draw': 100, 'prog_dim_0': 2})


ValueError: Unexpected dimensions in term prog

In [ ]:
ref_model.loo_compare()

UserWarning: Please run search before comparing submodels.